In [1]:
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt
from scipy.fftpack import fft, ifft
from sklearn import svm, decomposition
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import mean_absolute_error, accuracy_score, confusion_matrix

In [2]:
def plot_confusion_matrix(cm,classes,normalize=False,title='Confusion matrix',cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [3]:
ionosphere_data = pd.read_csv('C:\\Users\\seba_\\Documents\\ionosphere.csv', header=None)
ionosphere_data.head()

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,1,0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1.00000,0.03760,...,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300,1
1,1,0,1.00000,-0.18829,0.93035,-0.36156,-0.10868,-0.93597,1.00000,-0.04549,...,-0.26569,-0.20468,-0.18401,-0.19040,-0.11593,-0.16626,-0.06288,-0.13738,-0.02447,0
2,1,0,1.00000,-0.03365,1.00000,0.00485,1.00000,-0.12062,0.88965,0.01198,...,-0.40220,0.58984,-0.22145,0.43100,-0.17365,0.60436,-0.24180,0.56045,-0.38238,1
3,1,0,1.00000,-0.45161,1.00000,1.00000,0.71216,-1.00000,0.00000,0.00000,...,0.90695,0.51613,1.00000,1.00000,-0.20099,0.25682,1.00000,-0.32382,1.00000,0
4,1,0,1.00000,-0.02401,0.94140,0.06531,0.92106,-0.23255,0.77152,-0.16399,...,-0.65158,0.13290,-0.53206,0.02431,-0.62197,-0.05707,-0.59573,-0.04608,-0.65697,1


In [4]:
# reordenamiento aleatorio
ionosphere_data_shuffle = shuffle(ionosphere_data)

In [5]:
X = ionosphere_data_shuffle.iloc[:,0:33]

In [6]:
#pca = decomposition.PCA(n_components=8)
#pca.fit(X)
#F = pca.transform(X)

In [7]:
fc = np.absolute(fft(X))

In [8]:
y = ionosphere_data_shuffle.iloc[:,34]

In [9]:
xe, xv, ye, yv = train_test_split(fc[:,0:17], y, train_size = 0.6, test_size = 0.4)

In [10]:
C_range = [1, 10, 100, 1000]
gamma_range = np.logspace(-9, 3, 13)
kernel_range = ('linear', 'rbf', 'sigmoid', 'poly')
parameters = dict(gamma=gamma_range, C=C_range, kernel=kernel_range)

In [11]:
ionosphere_model = svm.SVC()

In [12]:
clf = GridSearchCV(ionosphere_model, parameters)
clf.fit(xe, ye)

GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'gamma': array([1.e-09, 1.e-08, 1.e-07, 1.e-06, 1.e-05, 1.e-04, 1.e-03, 1.e-02,
       1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]), 'C': [1, 10, 100, 1000], 'kernel': ('linear', 'rbf', 'sigmoid', 'poly')},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [13]:
print("The best parameters are %s with a score of %0.2f" % (clf.best_params_, clf.best_score_))

The best parameters are {'C': 10, 'gamma': 0.001, 'kernel': 'sigmoid'} with a score of 0.95


In [14]:
scores = cross_val_score(clf, xv, yv, cv=10)

In [15]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.92 (+/- 0.18)


In [16]:
#z = clf.predict(xv)

In [17]:
#print('La exactitud es de: %.2f' % accuracy_score(yv,z))

In [18]:
#cnf_matrix = confusion_matrix(yv, z)
#plt.figure()
#plot_confusion_matrix(cnf_matrix, classes=['good','bad'], normalize=False , title='Confusion matrix, without normalization')